In [1]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from bs4.element import Tag
from IPython.core.display import HTML
from tqdm import tqdm
import string

In [2]:
baseurl = 'https://www.lyricsfreak.com'

In [3]:
def getLyric(href):
  url = f'{baseurl}{href}'
  page = requests.get(url)
  html = page.text
  soup = BeautifulSoup(html, "lxml")
  text_div = soup.find(attrs={'class': 'lyrictxt js-lyrics js-share-text-content'})
  if text_div and text_div.get_text():
    return text_div.get_text().strip()
  return

In [4]:
def getArtistLyrics(href):
  url = f'{baseurl}{href}'
  page = requests.get(url)
  html = page.text
  soup = BeautifulSoup(html, "lxml")
  songs = {'song':[], 'link':[], 'text':[]}
  for tag in soup.find_all(attrs={'class': 'lf-link lf-link--primary'}):
    song_name = tag.string.split('Lyrics')[0].strip()
    song_href = tag['href']
    lyric = getLyric(song_href)
    songs['song'].append(song_name)
    songs['link'].append(song_href)
    songs['text'].append(lyric)
  return songs

In [5]:
def getSongsByArtistTop(top):
  url = f'{baseurl}/{top}_top.html'
  page = requests.get(url)
  html = page.text
  soup = BeautifulSoup(html, "lxml")
  all_df = pd.DataFrame()
  for tag in tqdm(soup.find_all(attrs={'class': 'lf-link lf-link--secondary'})):
    artist = tag.string.strip()
    href = tag['href']
    mapper = getArtistLyrics(href)
    current_df = pd.DataFrame(mapper)
    current_df['artist'] = artist
    current_df = current_df.reindex(columns=['artist', 'song', 'link', 'text'])
    all_df = pd.concat([all_df, current_df], ignore_index = True, axis = 0)
    all_df.to_csv(f'./engsongs_scrape_by_alphabets/{top}_artists.csv', index=False)
  return all_df

In [6]:
for top in ['g']:
  songs = getSongsByArtistTop(top)
  print(songs.head())

  0%|          | 0/30 [00:00<?, ?it/s]

In [ ]:
for x in ['c', 'd', 'e', 'f', 'g']